#Lasso Regression (L1 Regression)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Supervised Learning/csv files/Advertising.csv')

In [6]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [7]:
X = df.drop('sales',axis=1)
y= df['sales']

###Develope Features

In [8]:
from sklearn.preprocessing import PolynomialFeatures

In [9]:
p_conv = PolynomialFeatures(degree=3 , include_bias= False)

In [10]:
p_feat = p_conv.fit_transform(X)

###Train Test | Split

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test , y_train , y_test = train_test_split(p_feat,y,test_size=0.25,random_state = 101)

###Scaling

In [13]:
from sklearn.preprocessing import StandardScaler #Standardization

In [14]:
sc = StandardScaler()

In [15]:
sc_X_train = sc.fit_transform(X_train)
sc_X_test = sc.transform(X_test)

In [16]:
from sklearn.linear_model import LassoCV

In [17]:
model_lcv = LassoCV(eps= 0.1,n_alphas= 100,cv=5,max_iter=10000)

In [18]:
model_lcv.fit(sc_X_train,y_train)

LassoCV(cv=5, eps=0.1, max_iter=10000)

In [19]:
model_lcv.alpha_ #it used this alpha

np.float64(0.4955891288263106)

In [20]:
lcv_y_pred = model_lcv.predict(sc_X_test)

###Error Analysis

In [21]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [23]:
MAE = mean_absolute_error(y_test,lcv_y_pred)
MSE = mean_squared_error(y_test,lcv_y_pred)
RMSE = np.sqrt(MSE)

In [24]:
MAE

0.6241496332838827

In [25]:
MSE

0.9355589456162755

In [27]:
RMSE

np.float64(0.967242961006321)

In [26]:
model_lcv.coef_

array([1.03575888, 0.        , 0.        , 0.        , 3.79481506,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

##As we can see that in L2 regression we had 19 coefficients being used but now there is only 2 coefficients and that's the point we are trying to deal with baisedness of model so we will have to compromise with error

#Elastic Net

###Combination of L1(Lasso) and L2(Ridge)

In [28]:
from sklearn.linear_model import ElasticNetCV

In [29]:
en_model = ElasticNetCV(l1_ratio= [.01,0.1,0.5,0.9,1],eps=.1,max_iter=10000,cv=5)

In [30]:
en_model.fit(sc_X_train,y_train)

ElasticNetCV(cv=5, eps=0.1, l1_ratio=[0.01, 0.1, 0.5, 0.9, 1], max_iter=10000)

In [33]:
en_model.l1_ratio_ #l1_ratio =1 means only l1 panality got used , l1_ratio = 0 means only l2 penality is getting used and when value is in between then both is being used

np.float64(1.0)

In [36]:
en_model.alpha_

np.float64(0.4955891288263106)

In [37]:
y_en_pred = en_model.predict(sc_X_test)

In [38]:
MAE_en = mean_absolute_error(y_en_pred,y_test)

In [39]:
MSE_en = mean_squared_error(y_en_pred,y_test)

In [40]:
RMSE_en = np.sqrt(MSE_en)

In [41]:
MAE_en

0.6241496332838827

In [43]:
MSE_en

0.9355589456162755

In [44]:
RMSE_en

np.float64(0.967242961006321)

#Grid Search

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet #base model

In [47]:
base_elastic_model = ElasticNet()

In [54]:
param_grid = { 'alpha' : [0.1,1,5,10,50,100],
              'l1_ratio' : [0,0.1,0.2,0.3,0.5,0.9],
               'max_iter' : [1000]
}

In [60]:
grid_model = GridSearchCV(estimator=base_elastic_model,param_grid= param_grid,cv = 5,verbose = 1)

In [61]:
grid_model.fit(sc_X_train,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.073e+02, tolerance: 3.028e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.155e+02, tolerance: 3.276e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0, 0.1, 0.2, 0.3, 0.5, 0.9],
                         'max_iter': [1000]},
             verbose=1)

###Not sure why this  warning is coming but the results are still the expected one

In [62]:
grid_model.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=0.9)

In [63]:
y_pred = grid_model.predict(X_test)

In [64]:
np.sqrt(mean_squared_error(y_pred,y_test))

np.float64(3515725.647579289)

##TO DO - There is something wrong going on , gotta figure out later